# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fa266458610>
├── 'a' --> tensor([[-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506]])
└── 'x' --> <FastTreeValue 0x7fa2664580a0>
    └── 'c' --> tensor([[-1.7218, -1.0963,  1.0235,  0.6495],
                        [-0.7252,  0.8653, -0.4871, -0.9977],
                        [ 0.0175, -2.1153, -0.8417, -0.9102]])

In [4]:
t.a

tensor([[-0.8974, -0.1390,  0.0822],
        [ 0.7328,  1.3025,  1.2506]])

In [5]:
%timeit t.a

87.4 ns ± 0.954 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fa266458610>
├── 'a' --> tensor([[ 0.7980,  0.2013, -0.0286],
│                   [-0.8877, -0.3565, -2.0997]])
└── 'x' --> <FastTreeValue 0x7fa2664580a0>
    └── 'c' --> tensor([[-1.7218, -1.0963,  1.0235,  0.6495],
                        [-0.7252,  0.8653, -0.4871, -0.9977],
                        [ 0.0175, -2.1153, -0.8417, -0.9102]])

In [7]:
%timeit t.a = new_value

91.2 ns ± 0.633 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.8974, -0.1390,  0.0822],
               [ 0.7328,  1.3025,  1.2506]]),
    x: Batch(
           c: tensor([[-1.7218, -1.0963,  1.0235,  0.6495],
                      [-0.7252,  0.8653, -0.4871, -0.9977],
                      [ 0.0175, -2.1153, -0.8417, -0.9102]]),
       ),
)

In [10]:
b.a

tensor([[-0.8974, -0.1390,  0.0822],
        [ 0.7328,  1.3025,  1.2506]])

In [11]:
%timeit b.a

75.9 ns ± 0.459 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-0.1010,  0.1191,  0.8314],
               [-0.3752,  0.3506, -2.6274]]),
    x: Batch(
           c: tensor([[-1.7218, -1.0963,  1.0235,  0.6495],
                      [-0.7252,  0.8653, -0.4871, -0.9977],
                      [ 0.0175, -2.1153, -0.8417, -0.9102]]),
       ),
)

In [13]:
%timeit b.a = new_value

632 ns ± 3.88 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.06 µs ± 10.8 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.7 µs ± 92.6 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

164 µs ± 1.97 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

166 µs ± 881 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fa1965e8100>
├── 'a' --> tensor([[[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]],
│           
│                   [[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]],
│           
│                   [[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]],
│           
│                   [[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]],
│           
│                   [[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]],
│           
│                   [[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]],
│           
│                   [[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]],
│           
│                   [[-0.8974, -0.1390,  0.0822],
│                    [ 0.7328,  1.3025,  1.2506]]])
└── 'x' --> <FastTreeValue 0x7fa1965e8190>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

36.8 µs ± 218 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fa19e74de50>
├── 'a' --> tensor([[-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506],
│                   [-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506],
│                   [-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506],
│                   [-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506],
│                   [-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506],
│                   [-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506],
│                   [-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506],
│                   [-0.8974, -0.1390,  0.0822],
│                   [ 0.7328,  1.3025,  1.2506]])
└── 'x' --> <FastTreeValue 0x7fa19e74d490>
    └── 'c' --> tensor([[-1.7218, -1.0963,  1.0235,  0.6495],
                        [-0.7252,  0.8653, -0.4871, -0.9977],
                 

In [23]:
%timeit t_cat(trees)

34.9 µs ± 398 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

77 µs ± 375 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[-1.7218, -1.0963,  1.0235,  0.6495],
                       [-0.7252,  0.8653, -0.4871, -0.9977],
                       [ 0.0175, -2.1153, -0.8417, -0.9102]],
              
                      [[-1.7218, -1.0963,  1.0235,  0.6495],
                       [-0.7252,  0.8653, -0.4871, -0.9977],
                       [ 0.0175, -2.1153, -0.8417, -0.9102]],
              
                      [[-1.7218, -1.0963,  1.0235,  0.6495],
                       [-0.7252,  0.8653, -0.4871, -0.9977],
                       [ 0.0175, -2.1153, -0.8417, -0.9102]],
              
                      [[-1.7218, -1.0963,  1.0235,  0.6495],
                       [-0.7252,  0.8653, -0.4871, -0.9977],
                       [ 0.0175, -2.1153, -0.8417, -0.9102]],
              
                      [[-1.7218, -1.0963,  1.0235,  0.6495],
                       [-0.7252,  0.8653, -0.4871, -0.9977],
                       [ 0.0175, -2.1153, -0.8417, -0.9102]],

In [26]:
%timeit Batch.stack(batches)

96.1 µs ± 460 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[-1.7218, -1.0963,  1.0235,  0.6495],
                      [-0.7252,  0.8653, -0.4871, -0.9977],
                      [ 0.0175, -2.1153, -0.8417, -0.9102],
                      [-1.7218, -1.0963,  1.0235,  0.6495],
                      [-0.7252,  0.8653, -0.4871, -0.9977],
                      [ 0.0175, -2.1153, -0.8417, -0.9102],
                      [-1.7218, -1.0963,  1.0235,  0.6495],
                      [-0.7252,  0.8653, -0.4871, -0.9977],
                      [ 0.0175, -2.1153, -0.8417, -0.9102],
                      [-1.7218, -1.0963,  1.0235,  0.6495],
                      [-0.7252,  0.8653, -0.4871, -0.9977],
                      [ 0.0175, -2.1153, -0.8417, -0.9102],
                      [-1.7218, -1.0963,  1.0235,  0.6495],
                      [-0.7252,  0.8653, -0.4871, -0.9977],
                      [ 0.0175, -2.1153, -0.8417, -0.9102],
                      [-1.7218, -1.0963,  1.0235,  0.6495],
                   

In [28]:
%timeit Batch.cat(batches)

177 µs ± 1.42 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

407 µs ± 8.07 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
